In [4]:
#!pip install opytimizer

In [1]:
# Build simple ANN and compute the losses
import numpy as np
import pandas as pd
import torch
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from torch import optim
from torch.autograd import Variable
import random

#optymizer packages
from opytimizer import Opytimizer
from opytimizer.core.function import Function
from opytimizer.optimizers.swarm.pso import PSO
from opytimizer.spaces.search import SearchSpace

# Setting up a random seed
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)
torch.backends.cudnn.deterministic=True

In [2]:
# Loading digits dataset
digits = load_digits()

# Gathering samples and targets
X = digits.data
Y = digits.target

# Splitting the data
X_train, X_val, Y_train, Y_val = train_test_split(
    X, Y, test_size=0.3, random_state=42)

# Converting from numpy array to torch tensors
X_train = torch.from_numpy(X_train).float()
X_val = torch.from_numpy(X_val).float()
Y_train = torch.from_numpy(Y_train).long()

In [3]:
X_train.shape

torch.Size([1257, 64])

In [4]:
def fit(model, loss, opt, x, y):
    # Declaring initial variables
    x = Variable(x, requires_grad=False)
    y = Variable(y, requires_grad=False)

    # Resetting the gradient
    opt.zero_grad()

    # Performing the foward pass
    fw_x = model.forward(x)
    output = loss.forward(fw_x, y)

    # Performing backward pass
    output.backward()

    # Updating parameters
    opt.step()

    return output.item()
def set_random_seeds():
    torch.manual_seed(0)
    np.random.seed(0)
    random.seed(0)
    torch.backends.cudnn.deterministic=True


In [5]:
def predict(model, x_val):
    # Declaring validation variable
    x = Variable(x_val, requires_grad=False)

    # Performing backward pass with this variable
    output = model.forward(x)

    # Getting the index of the prediction
    y_val = output.data.numpy().argmax(axis=1)

    return y_val

In [6]:
def neural_network_test(learning_rate,momentum):
    set_random_seeds()
    # Instanciating the model
    model = torch.nn.Sequential()

    # Some model parameters
    n_features = 64
    n_hidden = 128
    n_classes = 10

    # Adding first linear layer
    model.add_module("linear_1", torch.nn.Linear(
        n_features, n_hidden, bias=False))

    # Followed by a sigmoid activation
    model.add_module("sigmoid_1", torch.nn.Sigmoid())

    # And finally a secondary linear layer
    model.add_module("linear_2", torch.nn.Linear(n_hidden, n_classes, bias=False))

    # Input variables
    batch_size = 64
    epochs = 100

    # Gathering parameters from Opytimizer
    # Pay extremely attention to their order when declaring due to their bounds
    #learning_rate = opytimizer[0][0]
    #momentum = opytimizer[1][0]

    # Declaring the loss function
    loss = torch.nn.CrossEntropyLoss(reduction='mean')

    # Declaring the optimization algorithm
    opt = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

    # Performing training loop
    for _ in range(epochs):
        # Initial cost as 0.0
        cost = 0.0

        # Calculating the number of batches
        num_batches = len(X_train) // batch_size

        # For every batch
        for k in range(num_batches):
            # Declaring initial and ending for each batch
            start, end = k * batch_size, (k + 1) * batch_size

            # Cost will be the loss accumulated from model's fitting
            cost += fit(model, loss, opt,
                        X_train[start:end], Y_train[start:end])

    # Return cost value
    return cost

In [7]:
learning_rate_range = [i/100 for i in range(0,100,10)]
momentum_range = [i/100 for i in range(0,100,10)]

In [8]:
cost_df = pd.DataFrame(index=learning_rate_range,columns=momentum_range)
for lr in learning_rate_range:
  for m in momentum_range:
    cost_df.loc[lr,m] = neural_network_test(lr,m)

In [9]:
print("Cost Values Data Frame :\n\n")
cost_df

Cost Values Data Frame :




,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0.0,44.3443,44.3443,44.3443,44.3443,44.3443,44.3443,44.3443,44.3443,44.3443,44.3443
0.1,0.26069,0.228831,0.1978,0.167449,0.138748,0.111438,0.0854143,0.0612762,0.0392288,0.0163804
0.2,0.111821,0.0985793,0.0857531,0.0732017,0.0614237,0.0498855,0.0384168,0.0277086,0.0180948,0.0142197
0.3,0.0703315,0.0623026,0.0540984,0.0465096,0.0388789,0.0315873,0.0244719,0.0180721,0.0116185,8.69925
0.4,0.0507036,0.0448642,0.0391157,0.0337243,0.0287311,0.0234398,0.0191246,0.0146669,0.0131176,13.1858
0.5,0.0405812,0.0359356,0.03202,0.0281717,0.0240475,0.0200787,0.0189903,0.0146119,2.46512,8.89051
0.6,0.035971,0.0324632,0.0288777,0.0257894,0.0238354,0.0207455,0.0206943,0.0622309,4.63159,18.4324
0.7,0.0351066,0.0304492,0.0287521,0.0263296,0.0253241,0.023906,0.0695451,1.28748,4.47118,54.422
0.8,0.0336559,0.0296337,0.0306941,0.0289439,0.0328993,0.0426726,1.24796,1.85688,4.57738,18.4781
0.9,0.0356803,0.033434,0.0344191,0.0396492,0.0405222,0.105292,4.09025,5.3406,7.59405,279.41


In [11]:
min_value = cost_df.min().min()
min_index = np.where(cost_df==min_value)
print("Minimum Cost Value is :",min_value)
print("Learning Rate :",learning_rate_range[min_index[0][0]])
print("Momentum :",momentum_range[min_index[1][0]])

Minimum Cost Value is : 0.011618530668783933
Learning Rate : 0.3
Momentum : 0.8


In [21]:
def neural_network(opytimizer):
    set_random_seeds()
    # Instanciating the model
    model = torch.nn.Sequential()

    # Some model parameters
    n_features = 64
    n_hidden = 128
    n_classes = 10

    # Adding first linear layer
    model.add_module("linear_1", torch.nn.Linear(
        n_features, n_hidden, bias=False))

    # Followed by a sigmoid activation
    model.add_module("sigmoid_1", torch.nn.Sigmoid())

    # And finally a secondary linear layer
    model.add_module("linear_2", torch.nn.Linear(n_hidden, n_classes, bias=False))

    # Input variables
    batch_size = 64
    epochs = 100

    # Gathering parameters from Opytimizer
    # Pay extremely attention to their order when declaring due to their bounds
    learning_rate = opytimizer[0][0]
    momentum = opytimizer[1][0]

    # Declaring the loss function
    loss = torch.nn.CrossEntropyLoss(reduction='mean')

    # Declaring the optimization algorithm
    opt = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

    # Performing training loop
    for _ in range(epochs):
        # Initial cost as 0.0
        cost = 0.0

        # Calculating the number of batches
        num_batches = len(X_train) // batch_size

        # For every batch
        for k in range(num_batches):
            # Declaring initial and ending for each batch
            start, end = k * batch_size, (k + 1) * batch_size

            # Cost will be the loss accumulated from model's fitting
            cost += fit(model, loss, opt,
                        X_train[start:end], Y_train[start:end])

    # Predicting samples from evaluating set
    return cost


# Creating Function's object
f = Function(pointer=neural_network)

# Number of agents
n_agents = 10

# Number of decision variables
n_variables = 2

# Number of running iterations
n_iterations = 20

# Lower and upper bounds (has to be the same size as n_variables)
lower_bound = (0, 0)
upper_bound = (1, 1)

# Creating the SearchSpace class
s = SearchSpace(n_agents=n_agents, n_iterations=n_iterations,
                n_variables=n_variables, lower_bound=lower_bound,
                upper_bound=upper_bound)

# Hyperparameters for the optimizer
hyperparams = {
    'w': 0.9,
    'c1': 1.1,
    'c2': 1.9
}

# Creating PSO's optimizer
p = PSO(hyperparams=hyperparams)

# Finally, we can create an Opytimizer class
o = Opytimizer(space=s, optimizer=p, function=f)

2020-08-28 09:11:46,483 - opytimizer.core.function — INFO — Creating class: Function.
2020-08-28 09:11:46,484 - opytimizer.core.function — INFO — Class created.
2020-08-28 09:11:46,485 - opytimizer.core.function — DEBUG — Function: neural_network | Constraints: [] | Penalty: 0.0 | Built: True
2020-08-28 09:11:46,487 - opytimizer.spaces.search — INFO — Overriding class: Space -> SearchSpace.
2020-08-28 09:11:46,488 - opytimizer.core.space — DEBUG — Running private method: build().
2020-08-28 09:11:46,490 - opytimizer.core.space — DEBUG — Running private method: create_agents().
2020-08-28 09:11:46,493 - opytimizer.core.space — DEBUG — Agents: 10 | Size: (2, 1) | Iterations: 20 | Lower Bound: [0 0] | Upper Bound: [1 1] | Built: True.
2020-08-28 09:11:46,495 - opytimizer.spaces.search — DEBUG — Running private method: initialize_agents().
2020-08-28 09:11:46,499 - opytimizer.spaces.search — DEBUG — Agents initialized.
2020-08-28 09:11:46,500 - opytimizer.spaces.search — INFO — Class overr

In [22]:
# Running the optimization task
history = o.start()

2020-08-28 09:11:52,405 - opytimizer.opytimizer — INFO — Starting optimization task.


100%|██████████| 20/20 [03:11<00:00,  9.57s/it, fitness=2.46e-6]

2020-08-28 09:15:14,925 - opytimizer.opytimizer — INFO — Optimization task ended.
2020-08-28 09:15:14,926 - opytimizer.opytimizer — INFO — It took 202.51922082901 seconds.


In [23]:
print(history)


Iteration 1/20

Position: [[0.4218511272534194], [0.6464674497746838]] | Fitness: 0.015936124429572374

Iteration 2/20

Position: [[0.4218511272534194], [0.6464674497746838]] | Fitness: 0.015936124429572374

Iteration 3/20

Position: [[0.42949719615890497], [0.7887676110456674]] | Fitness: 0.012992645351914689

Iteration 4/20

Position: [[0.42949719615890497], [0.7887676110456674]] | Fitness: 0.012992645351914689

Iteration 5/20

Position: [[0.4562522586747916], [0.7479889701223974]] | Fitness: 0.012473740876885131

Iteration 6/20

Position: [[0.23985693081839723], [0.8638426830374502]] | Fitness: 0.011275383032625541

Iteration 7/20

Position: [[0.045101135747642324], [0.9681110246609976]] | Fitness: 0.011225399110116996

Iteration 8/20

Position: [[0.0010277848853600668], [1.0]] | Fitness: 0.0005167698965475864

Iteration 9/20

Position: [[0.0010277848853600668], [1.0]] | Fitness: 0.0005167698965475864

Iteration 10/20

Position: [[0.0010277848853600668], [1.0]] | Fitness: 0.0005167

In [24]:
neural_network_test(0.008612887409813821,1.0)

2.458593441723167e-06